In [1]:
import os
import azureml.core
from azureml.core import Workspace, Experiment, Datastore
from azureml.widgets import RunDetails

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.22.0


In [10]:
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter


print("Pipeline SDK-specific imports completed")

Pipeline SDK-specific imports completed


In [3]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Default datastore
def_blob_store = ws.get_default_datastore() 
# The following call GETS the Azure Blob Store associated with your workspace.
# Note that workspaceblobstore is **the name of this store and CANNOT BE CHANGED and must be used as is** 
def_blob_store = Datastore(ws, "workspaceblobstore")
print("Blobstore's name: {}".format(def_blob_store.name))

amls-databricks
jp-databricks
centralus
29f97392-4349-4e87-9ad4-35ae6c1ec2e7
Blobstore's name: workspaceblobstore


In [4]:

cts = ws.compute_targets
for ct in cts:
    print(ct)

databricksprod
aml-compute
jp-aml


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

aml_compute_target = "aml-compute"
try:
    aml_compute = AmlCompute(ws, aml_compute_target)
    print("found existing compute target.")
except ComputeTargetException:
    print("creating new compute target")
    
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2",
                                                                min_nodes = 1, 
                                                                max_nodes = 4)    
    aml_compute = ComputeTarget.create(ws, aml_compute_target, provisioning_config)
    aml_compute.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
print("Azure Machine Learning Compute attached")

found existing compute target.
Azure Machine Learning Compute attached


In [16]:
# Uses default values for PythonScriptStep construct.

source_directory = './scripts'
print('Source directory for the step is {}.'.format(os.path.realpath(source_directory)))


pipeline_param = PipelineParameter(name="pipeline_arg", default_value="default_val")

step1 = PythonScriptStep(name="script_step",
                         script_name="script.py",
                         arguments=["--pipeline_arg", pipeline_param],
                         compute_target=aml_compute, 
                         source_directory=source_directory,
                         allow_reuse=True)
print("Step1 created")

Source directory for the step is /mnt/batch/tasks/shared/LS_root/mounts/clusters/jp-aml/code/Users/joplum/pipeline-param/scripts.
Step1 created


In [17]:
steps = [step1]
print("Step lists created")

Step lists created


In [18]:
pipeline1 = Pipeline(workspace=ws, steps=steps)
print ("Pipeline is built")

Pipeline is built


In [19]:
pipeline1.validate()
print("Pipeline validation complete")

Step script_step is ready to be created [45839d19]
Pipeline validation complete


In [20]:
pipeline_run1 = Experiment(ws, 'pipeline1-submit').submit(pipeline1, regenerate_outputs=False, \
                                                          pipeline_parameters={'pipeline_arg':'hello pipeline param'})
print("Pipeline is submitted for execution")

Created step script_step [45839d19][f57badf5-4734-4d1a-be08-a15384633529], (This step will run and generate new outputs)
Submitted PipelineRun 4719de30-9137-46a2-b0c1-9133aac48420
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/pipeline1-submit/runs/4719de30-9137-46a2-b0c1-9133aac48420?wsid=/subscriptions/29f97392-4349-4e87-9ad4-35ae6c1ec2e7/resourcegroups/jp-databricks/workspaces/amls-databricks
Pipeline is submitted for execution


In [21]:
RunDetails(pipeline_run1).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [23]:
published_pipeline = pipeline1.publish(name="ParamPipeline", description="Pipeline to test parameters", continue_on_step_failure=True)
published_pipeline

Name,Id,Status,Endpoint
ParamPipeline,0d914357-6b06-4955-96ea-849e0e8b42ae,Active,REST Endpoint
